# Analyse von Veröffentlichungen mit KI Bezug von CSRankings.org

## Rohdaten

Die Datensätze wurden mit dem Code unter https://github.com/emeryberger/CSrankings erzeugt.

Erstellungsdatum: 06.01.2019

In [1]:
import pandas as pd

affiliations = pd.read_csv("./data/faculty-affiliations.csv")
universities = pd.read_csv("./data/country-info.csv")
articles = pd.read_json("./data/articles.json")

## Hochschulen
Filtern Europäischer Hochschulen (durch Attribut in Datensatz)

In [2]:
european_universities = universities[universities["region"] == "europe"]

Ausgabe der europäischen Universitäten zur manuellen Filterung hessischer/deutscher Einrichtungen

In [3]:
pd.options.display.max_rows = -1 # do not abbreviate the list
european_universities

,institution,region
0,Aalto University,europe
1,Aarhus University,europe
3,Ariel University,europe
4,AUEB,europe
7,Bar-Ilan University,europe
9,Ben-Gurion University of the Negev,europe
11,CWI,europe
13,CISPA Helmholtz Center i.G.,europe
14,Charles University,europe
20,Cyprus University of Technology,europe


Manuelle Identifizierung Deutscher Hochschulen und Ausgabe zur Filterung Hessischer Hochschulen

In [4]:
german_universities = european_universities.loc[[13, 72, 78, 94, 95, 105, 106, 107, 109, 111, 142, 147, 159, 184, 199, 200]]
german_universities

,institution,region
13,CISPA Helmholtz Center i.G.,europe
72,LMU Munich,europe
78,Max Planck Institute,europe
94,Ruhr-University Bochum,europe
95,RWTH Aachen,europe
105,TU Berlin,europe
106,TU Braunschweig,europe
107,TU Darmstadt,europe
109,TU Dresden,europe
111,TU Munich,europe


Manuelle Identifizierung Hessischer Hochschulen

In [5]:
hessian_universities = german_universities.loc[[107,]]

## Artikel
Aussortieren doppelter Einträge (nach Titel)

In [6]:
before = len(articles)
articles = articles.drop_duplicates(subset="title")
after = len(articles)

"{} Duplikate entfernt".format(before - after)

'40656 Duplikate entfernt'

Auswählen der Artikel aus "KI"-Konferenzen

In [7]:
ai_articles = articles[articles.area.isin(["aaai", "ijcai", "cvpr", "eccv", "iccv", "icml", "kdd", "nips", "acl", "emnlp", "naacl", "sigr", "www"])]
"KI-Artikel insgesamt: {}".format(len(ai_articles))

'KI-Artikel insgesamt: 30623'

Matching der KI-Artikel mit den deutschen Hochschulen

In [8]:
german_ai_articles = ai_articles.join(german_universities.set_index("institution"), on="institution", how="inner")

In [9]:
"KI-Artikel von deutschen Universitäten: {}".format(len(german_ai_articles))

'KI-Artikel von deutschen Universitäten: 1162'

Einfügen einer Spalte `from_hessen` in die Tabelle der Deutschen KI-Artikel um Artikel Hessischer Hochschulen zu kennzeichen

In [10]:
german_ai_articles["from_hessen"] = german_ai_articles["institution"].isin(hessian_universities["institution"])
"KI-Artikel hessicher Universitäten: {}".format(len(german_ai_articles[german_ai_articles["from_hessen"]==True]))

'KI-Artikel hessicher Universitäten: 132'

## Visualiserung/Export

Export der Liste deutsche KI-Artikel nach `publications.csv`

In [11]:
german_ai_articles.to_csv("publications.csv")

Säulindiagramm der Anzahl Deutscher und Hessischer Beiträge zu "KI-Konferenzen"

In [12]:
import plotly.offline as py
import plotly.graph_objs as go

vc = german_ai_articles[german_ai_articles["from_hessen"]==False]["year"].value_counts()
vc_hessen = german_ai_articles[german_ai_articles["from_hessen"]==True]["year"].value_counts()

data = [
    go.Bar(
        x=vc.index.values,
        y=vc.values,
        name = "Deutschland Rest"
    ),
    go.Bar(
        x=vc_hessen.index.values,
        y=vc_hessen.values,
        name="Hessen"
    )
]

layout = go.Layout(
    barmode='stack',
    title='Beiträge zu KI-Konferenzen Deutscher Hochschulen nach Jahren'
)

fig = go.Figure(data=data, layout=layout)
chart = py.plot(fig, filename="publications.html")

![chart](publications.png)